In [1]:
import polars as pl
from blueskyranker import Fetcher
from blueskyranker import TopicRanker

In [2]:
# Download data:
fetcher = Fetcher()
result = fetcher.fetch()
print(result)



Posts fetched (all handles): 0post [00:00, ?post/s]
Handles:   0%|          | 0/4 [00:00<?, ?handle/s]

Posts fetched (all handles): 1post [00:00,  2.00post/s]

[news-flows-nl.bsky.social] pages — last: 2025-08-19T20:45:00+00:00 — within window: yes: : 0page [00:00, ?page/s]

Posts fetched (all handles): 101post [00:01, 65.83post/s]

[news-flows-nl.bsky.social] pages — last: 2025-08-19T16:23:43+00:00 — within window: yes: : 1page [00:01,  2.69page/s]

[news-flows-nl.bsky.social] pages — last: 2025-08-19T16:23:43+00:00 — within window: yes: : 2page [00:01,  1.18page/s]

[news-flows-nl.bsky.social] pages — last: 2025-08-19T12:51:17+00:00 — within window: no: : 2page [00:02,  1.18page/s] 

[news-flows-nl.bsky.social] pages — last: 2025-08-19T12:51:17+00:00 — within window: no: : 3page [00:02,  1.29page/s]


Posts fetched (all handles): 200post [00:02, 65.83post/s]
Handles:  25%|██▌       | 1/4 [00:02<00:07,  2.53s/handle]✅ DONE news-flows-nl.bsky.social: added 199 new posts (2025-08-19T16

In [2]:
# load the data that keys()we just downloaded
data = pl.read_csv('news-flows-nl_bsky_social_author_feed.csv')

In [3]:
myranker = TopicRanker(returnformat='dataframe', method = 'networkclustering-tfidf', descending=False)
ranking = myranker.rank(data)


09:07:01 DEBUG    Creating cosine similarity matrix...
09:07:02 DEBUG    Removing all entries below a threshold of 0.2
09:07:02 DEBUG    The new matrix is 99.71% sparse
09:07:02 DEBUG    Creating a graph
09:07:03 DEBUG    Apply network clustering using the Leiden Algorithm
09:07:04 DEBUG    The 3306 items were grouped into 1075 clusters.


In [7]:
# We can inspect the ranking...
ranking

uri,cid,author_handle,author_did,indexedAt,createdAt,text,reply_root_uri,reply_parent_uri,is_repost,like_count,repost_count,reply_count,quote_count,news_title,news_description,news_uri,cluster,cluster_like_count,cluster_reply_count,cluster_quote_count,cluster_repost_count,cluster_size,cluster_engagement_count,cluster_engagement_rank
str,str,str,str,str,str,str,null,null,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64
"""at://did:plc:toz4no26o2x4vsbum…","""bafyreia3vnna2s3vyuzpnhyuytk3m…","""news-flows-nl.bsky.social""","""did:plc:toz4no26o2x4vsbum7cp4b…","""2025-08-19T13:38:29.610Z""","""2025-08-19T13:18:25.000000Z""","""Zelensky 'overleeft' het Witte…",null,null,0,0,0,0,0,"""Zelensky 'overleeft' het Witte…","""Na twee toppen in Alaska en Wa…","""https://www.nu.nl/spanningen-o…",0,0,0,1,0,203,1,1054
"""at://did:plc:toz4no26o2x4vsbum…","""bafyreide3xsf72ya264xlon2rn3kc…","""news-flows-nl.bsky.social""","""did:plc:toz4no26o2x4vsbum7cp4b…","""2025-08-19T13:38:25.112Z""","""2025-08-19T13:13:00.000000Z""","""Zelensky gaf Trump golfclub en…",null,null,0,0,0,0,0,"""LIVE Oorlog Oekraïne | Zelensk…","""Zelensky heeft Trump bij zijn …","""https://www.ad.nl/buitenland/l…",0,0,0,1,0,203,1,1054
"""at://did:plc:toz4no26o2x4vsbum…","""bafyreicpuf6s4nd7d354pz253qps7…","""news-flows-nl.bsky.social""","""did:plc:toz4no26o2x4vsbum7cp4b…","""2025-08-19T14:35:04.107Z""","""2025-08-19T12:41:39.000000Z""","""Trump hoopvol over overeenkoms…",null,null,0,0,0,0,0,"""Trump hoopvol over overeenkoms…","""Oorlog in Oekraïne: In dit blo…","""https://www.nrc.nl/nieuws/2025…",0,0,0,1,0,203,1,1054
"""at://did:plc:toz4no26o2x4vsbum…","""bafyreidqgmsjqocgmjv2gpjk4wz4j…","""news-flows-nl.bsky.social""","""did:plc:toz4no26o2x4vsbum7cp4b…","""2025-08-19T12:48:12.207Z""","""2025-08-19T12:22:00.000000Z""","""Top in Witte Huis een succes? …",null,null,0,0,0,0,0,"""Top in Witte Huis een succes? …","""In het Westen overheerst voorz…","""https://www.volkskrant.nl/nieu…",0,0,0,1,0,203,1,1054
"""at://did:plc:toz4no26o2x4vsbum…","""bafyreihgjvu3gai4qlyishjbm5keu…","""news-flows-nl.bsky.social""","""did:plc:toz4no26o2x4vsbum7cp4b…","""2025-08-19T12:08:10.207Z""","""2025-08-19T11:57:05.000000Z""","""Russische minister Lavrov: 'Ge…",null,null,0,0,0,0,0,"""Russische minister Lavrov: 'Ge…","""De Russische buitenlandministe…","""https://www.rtl.nl/nieuws/buit…",0,0,0,1,0,203,1,1054
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""at://did:plc:toz4no26o2x4vsbum…","""bafyreiezsaerj2kzqlk2eew6xmghl…","""news-flows-nl.bsky.social""","""did:plc:toz4no26o2x4vsbum7cp4b…","""2025-08-19T17:11:57.717Z""","""2025-08-19T16:52:57.000000Z""","""Hamas akkoord met staakt-het-v…",null,null,0,0,0,0,0,"""Hamas akkoord met staakt-het-v…","""Oorlog in Gaza: Hamas stemt in…","""https://www.nrc.nl/nieuws/2025…",18,1,1,0,0,34,2,1071
"""at://did:plc:toz4no26o2x4vsbum…","""bafyreigu6i2r5w53xm7zhj65mretp…","""news-flows-nl.bsky.social""","""did:plc:toz4no26o2x4vsbum7cp4b…","""2025-08-14T23:26:49.604Z""","""2025-08-14T23:15:06.000000Z""","""Philips investeert ruim 150 mi…",null,null,0,0,0,0,0,"""Philips investeert ruim 150 mi…","""Philips gaat ruim 150 miljoen …","""https://www.ad.nl/economie/phi…",155,0,1,0,1,3,2,1072
"""at://did:plc:toz4no26o2x4vsbum…","""bafyreigm4ra2ry3p5lbv6jktcd3un…","""news-flows-nl.bsky.social""","""did:plc:toz4no26o2x4vsbum7cp4b…","""2025-08-14T09:04:00.207Z""","""2025-08-14T08:53:18.000000Z""","""De Amerikaanse talentenmagneet…",null,null,0,1,1,0,0,"""De Amerikaanse talentenmagneet…","""Wetenschapsbeleid: De VS golde…","""https://www.nrc.nl/nieuws/2025…",811,1,0,0,1,1,2,1073


In [ ]:
# and if we want to, we can now post it to the server!
myranker.post(test=True)
